<a href="https://colab.research.google.com/github/Murcicrum/Scrapers-de-inmobiliarias/blob/main/scraper_mercadolibre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
1  '''
EN PROCESO
  '''

Imports

In [ ]:
import time
import requests
from bs4 import BeautifulSoup as bs

#Levanto publicaciones

Defino los urls de las busquedas

In [ ]:
url_search_deptos = 'https://inmuebles.mercadolibre.com.ar/departamentos/alquiler/capital-federal/_Desde_'
url_search_casas  = 'https://inmuebles.mercadolibre.com.ar/casas/alquiler/capital-federal/_Desde_'
url_search_phs    = 'https://inmuebles.mercadolibre.com.ar/ph/alquiler/capital-federal/_Desde_'


#####Inspección a mano

In [ ]:
#from pprint import PrettyPrinter as pp
import pprint
pp = pprint.PrettyPrinter()

In [ ]:
response = requests.get( url_search_deptos + str(1) , 
                         headers={'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:91.0) Gecko/20100101 Firefox/91.0'} )
soup = bs(response.content)
#def get_links(url_search):
print( response.status_code, len(soup.prettify()) )
#print('RES_H:', response.headers)
print('REQ_H:', response.request.headers)

200 720967
REQ_H: {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:91.0) Gecko/20100101 Firefox/91.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Cookie': '_d2id=be3e412f-9562-441b-bdc4-c805992369e3-n; _csrf=wEjpFlhgfdCI8r5H-A8pQTgF; cdnx=1'}


In [ ]:
#200 733251
{'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:91.0) Gecko/20100101 Firefox/91.0', 
 'Accept-Encoding': 'gzip, deflate', 
 'Accept': '*/*', 
 'Connection': 'keep-alive', 
 'Cookie': '_d2id=b8d0863c-dd6c-4d52-8533-302e35ffa773-n; _csrf=LLrq1rJBJCfGaG_3AGQFJEIO; cdnx=1'}        #esto cambia siempre

#200 721493
{'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:91.0) Gecko/20100101 Firefox/91.0', 
 'Accept-Encoding': 'gzip, deflate', 
 'Accept': '*/*', 
 'Connection': 'keep-alive', 
 'Cookie': '_d2id=c258bfbf-8996-4bee-b91e-4c9342ae934c-n; _csrf=rzwXss0pgUYTbFofBIUTxZJs; cdnx=1'}

#200 721490
{'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:91.0) Gecko/20100101 Firefox/91.0', 
 'Accept-Encoding': 'gzip, deflate', 
 'Accept': '*/*', 
 'Connection': 'keep-alive', 
 'Cookie': '_d2id=4fa15557-a82c-4609-a359-09a194585e9c-n; _csrf=nv23EAiCPvliRsTpCFXF8sy0; cdnx=1'}

{'Accept': '*/*',
 'Accept-Encoding': 'gzip, deflate',
 'Connection': 'keep-alive',
 'Cookie': '_d2id=4fa15557-a82c-4609-a359-09a194585e9c-n; _csrf=nv23EAiCPvliRsTpCFXF8sy0; cdnx=1',
 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:91.0) Gecko/20100101 Firefox/91.0'}

96-48 723643, 722999, 724161, 724682, 724009 \\
50-50 738032, 737027, 737091, 740797

In [ ]:
#import re
tag_deptos = soup.find_all( name='a', attrs={'title':'Departamento en alquiler','class': 'ui-search-link'} )  # No bastaría con '^ui-search-link' ??
tag_deptos[:10]

[<a class="ui-search-link" href="https://departamento.mercadolibre.com.ar/MLA-1103660441-las-heras-y-pueyrredon-3-ambientes-_JM#position=1&amp;search_layout=grid&amp;type=item&amp;tracking_id=9f3a4a04-8d86-4041-99ab-530f58bfd03c" target="_blank" title="Departamento en alquiler"><div class="carousel-container arrow-visible"><div class="slick-initialized slick-slider"><div class="slick-list" role="presentation"><div class="slick-track"><div class="slick-slide slick-active" data-index="0" style="outline:none" tabindex="-1"><img alt="Departamento en alquiler" class="ui-search-result-image__element lazy-loadable" data-src="https://http2.mlstatic.com/D_NQ_NP_796950-MLA47530833550_092021-W.webp" height="378" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" width="284"/></div></div></div></div></div></a>,
 <a class="ui-search-result__content ui-search-link" href="https://departamento.mercadolibre.com.ar/MLA-1103660441-las-heras-y-pueyrredon-3-ambientes-_JM#p

In [ ]:
urls = [ t['href'] for t in tag_deptos ]
len(tag_deptos), len(set(tag_deptos)), len(urls), len(set(urls))

(96, 96, 96, 48)

#####Armo scraper que guarde los links de las publicaciones para un url_search dado

In [ ]:
def get_links(url_search:list, category:str, N:int) -> list:
  '''
  Devuelve lista con los links a publicaciones de alquileres, dado:
    url_search  string de búsqueda con el formato que especifica numero de página
    category    puede ser 'Departamento', 'Casa', o 'Ph'
    N           numero de páginas a recorrer
  '''
  urls = []

  for n in range(N):
    
    url_request = url_search + str( n*48+1)
    response = requests.get( url_request )
    
    print('Sacando links de página ', n, '\tURL ', url_request)
    
    if response.status_code != 200:
      print('Request malió sal.', '\tStatus code', response.status_code)
      continue

    soup = bs( response.content )
    tags = soup.find_all( name='a', attrs={'title': 'Departamento en alquiler', 'class':'ui-search-link'} )
    urls += [ t['href'] for t in tags ]
  urls = list(set(urls))
  print(f'Se extrajeron {len(urls)} links')
  return urls

In [ ]:
url_deptos = get_links(url_search_deptos, 'Departamento', 5)

Sacando links de página  0 	URL  https://inmuebles.mercadolibre.com.ar/departamentos/alquiler/capital-federal/_Desde_1
Sacando links de página  1 	URL  https://inmuebles.mercadolibre.com.ar/departamentos/alquiler/capital-federal/_Desde_49
Sacando links de página  2 	URL  https://inmuebles.mercadolibre.com.ar/departamentos/alquiler/capital-federal/_Desde_97
Sacando links de página  3 	URL  https://inmuebles.mercadolibre.com.ar/departamentos/alquiler/capital-federal/_Desde_145
Sacando links de página  4 	URL  https://inmuebles.mercadolibre.com.ar/departamentos/alquiler/capital-federal/_Desde_193
Se extrajeron 242 links


In [ ]:
a = [45,123,745,12,12,1,53,73,2,352,2,2,51,3,123]
list(set(a))

[352, 1, 2, 3, 745, 73, 12, 45, 51, 53, 123]

In [ ]:
help(set)

Help on class set in module builtins:

class set(object)
 |  set() -> new empty set object
 |  set(iterable) -> new set object
 |  
 |  Build an unordered collection of unique elements.
 |  
 |  Methods defined here:
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __contains__(...)
 |      x.__contains__(y) <==> y in x.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iand__(self, value, /)
 |      Return self&=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __ior__(self, value, /)
 |      Return self|=value.
 |  
 |  __isub__(self, value, /)
 |      Return self-=value.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __ixor__(self, value, /)
 |      Re

In [ ]:
url_deptos[:5]

['https://departamento.mercadolibre.com.ar/MLA-1102045080-departamento-almagro-_JM#position=9&search_layout=stack&type=item&tracking_id=9fc7daf8-b34b-4194-88a0-b3efc2c66a53',
 'https://departamento.mercadolibre.com.ar/MLA-1100261146-departamento-palermo-_JM#position=48&search_layout=grid&type=item&tracking_id=cc5b249a-d0c6-4172-b9ae-a0e3ce0f740c',
 'https://departamento.mercadolibre.com.ar/MLA-932689000-alquiler-dpto-2-ambientes-a-metros-plaza-almagro-_JM#position=43&search_layout=grid&type=item&tracking_id=f487ea98-7e2d-46f4-b0ef-c57109de86f9',
 'https://departamento.mercadolibre.com.ar/MLA-1100777798-departamento-once-_JM#position=33&search_layout=grid&type=item&tracking_id=a746e2c6-e28c-434a-aeb9-8559569489f1',
 'https://departamento.mercadolibre.com.ar/MLA-930583860-luminoso-dueno-todo-electricovalcon-_JM#position=43&search_layout=grid&type=item&tracking_id=cc5b249a-d0c6-4172-b9ae-a0e3ce0f740c']

In [ ]:
url_casas = get_links(url_search_casas, 'Casa', 10)

Sacando links de página  0
Sacando links de página  1
Sacando links de página  2
Sacando links de página  3
Sacando links de página  4
Sacando links de página  5
Sacando links de página  6
Sacando links de página  7
Sacando links de página  8
Sacando links de página  9
Se extrajeron 195 links


In [ ]:
url_casas[:5]

['https://casa.mercadolibre.com.ar/MLA-931815128-casa-slote-propio-en-alquiler-comercial-caballito-norte-_JM#position=31&search_layout=grid&type=item&tracking_id=f7dfb0c4-9193-4c27-99e7-736d87a25231',
 'https://casa.mercadolibre.com.ar/MLA-911133996-alquiler-de-casa-de-5-ambientes-con-garage-en-p-avellaneda-_JM#position=28&search_layout=stack&type=item&tracking_id=10299457-1d56-420d-ae05-fe84f2e34175',
 'https://casa.mercadolibre.com.ar/MLA-899863547-casa-en-palermo-_JM#position=44&search_layout=stack&type=item&tracking_id=10299457-1d56-420d-ae05-fe84f2e34175',
 'https://casa.mercadolibre.com.ar/MLA-1101262887-oppel-casa-en-alquiler-belgrano-capital-federal-cod-_JM#position=30&search_layout=grid&type=item&tracking_id=f7dfb0c4-9193-4c27-99e7-736d87a25231',
 'https://casa.mercadolibre.com.ar/MLA-1101864339-casa-lde-nunez-_JM#position=42&search_layout=grid&type=item&tracking_id=587a709f-e8b4-4521-89b8-2d29750fa16e']

In [ ]:
url_phs = get_links(url_search_phs, 'Ph', 10)

Sacando links de página  0
Sacando links de página  1
Sacando links de página  2
Sacando links de página  3
Sacando links de página  4
Sacando links de página  5
Sacando links de página  6
Sacando links de página  7
Sacando links de página  8
Sacando links de página  9
Se extrajeron 224 links


In [ ]:
url_phs[:5]

['https://inmueble.mercadolibre.com.ar/MLA-1100460800-alquiler-ph-tipo-casa-10-ambientes-parquechacabuco-580m2-_JM#position=35&search_layout=stack&type=item&tracking_id=3b979732-4a9a-4e95-a99a-0744137e81a2',
 'https://inmueble.mercadolibre.com.ar/MLA-930734454-tipo-casa-en-triplex-con-cpchera-trerraza-_JM#position=47&search_layout=stack&type=item&tracking_id=3b979732-4a9a-4e95-a99a-0744137e81a2',
 'https://inmueble.mercadolibre.com.ar/MLA-935702890-lascano-al-4700-ph-1-amb-divisible-con-patio-_JM#position=37&search_layout=stack&type=item&tracking_id=72004a38-733a-4da2-9a60-4cf7eced9bbc',
 'https://inmueble.mercadolibre.com.ar/MLA-927422371-ph-caballito-_JM#position=37&search_layout=stack&type=item&tracking_id=3b979732-4a9a-4e95-a99a-0744137e81a2',
 'https://inmueble.mercadolibre.com.ar/MLA-1102825656-ph-palermo-_JM#position=23&search_layout=grid&type=item&tracking_id=fd80cf05-5fc9-418c-a28d-653fa424c251']


#Levanto datos